# Use this script to convert rich txt files (i.e. txt with markup annotation) to the MVN xml-standard

## add_numbers:

In [43]:
import re
from copy import deepcopy
# input: txt, af en toe nummer

#nummer aan elke lijn toevoegen

def add_numbers(text_file):
    output = []
    
    with open(text_file, 'r', encoding='utf-8') as input_file: # open het bestand
        input = input_file.readlines() # lees het in, maak een lijst met 1 item = 1 lijn in de txt

    last_line = 0
    for index, line in enumerate(input): # ga door elke lijn, en tel ook op welke lijn je zit (= index)
        if not re.match(r'^&', line) and not re.match(r'^_', line) and not re.match(r'\s*\n', line): # lijnen met & of _ negeren, alsook lege lijnen negeren
            if re.match(r'^[0-9]', line):
                try:
                    last_line = int(re.sub(' .*$', '', line)) # herken het nummer als er eentje staat, dat wordt nu het nummer van de laatste lijn
                    line_number = str(last_line).zfill(3)
                    line_no_number = (re.sub('^[0-9]* ', '', line)).strip()
                    if line_no_number:
                        output.append([line_number, line_no_number])
                except: # de lijn-nummer gewoon +1 doen als de 'try' niet lukt (dus als het nummer niet herkend wordt)
                    last_line += 1
            if not re.match(r'^[0-9]', line): #als er geen nummer aan de start van de regel staat
                last_line += 1 # 1 meer dan de laatste lijn
                line_number = str(last_line).zfill(3)
                line = line.strip()
                if line:
                    output.append([line_number, line])  
        else:
            output.append(['no_content_line', line.strip()])
    return output


## script Martijn

In [11]:
import string, re
import glob, os

rubric = re.compile(r"(\<)([a-zA-Z]+)(\>)")
boundaries = re.compile(r"(\[|\]|\(|\)|\%|\$|\=|\{|\}|\*|\£)")
endings = re.compile(r"(\)|\]|\%|\$|\=|\}|\*|\£)$")
beginnings = re.compile(r"$\(|\[|\{")

In [12]:
# path to the rich txt files
txt_path = '../data/rich_txt_martijn/'

file_paths = []

for root, dirs, files in os.walk(txt_path):
    for file in files:
        if file.endswith("checkpoint.txt"): # disregard txt files that are Notebook Checkpoints
            continue
        if file.endswith(".txt"):
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
            print(file_path)

../data/rich_txt_martijn/Uitgave_A\TxT_A.txt
../data/rich_txt_martijn/Uitgave_Ant\TxT_Ant.txt
../data/rich_txt_martijn/Uitgave_B\TxT_B.txt
../data/rich_txt_martijn/Uitgave_Br\TxT_Br.txt
../data/rich_txt_martijn/Uitgave_C\TxT_C.txt
../data/rich_txt_martijn/Uitgave_D\TxT_D.txt
../data/rich_txt_martijn/Uitgave_D2\TxT_D2.txt
../data/rich_txt_martijn/Uitgave_E\TxT_E.txt
../data/rich_txt_martijn/Uitgave_F\TxT_F.txt
../data/rich_txt_martijn/Uitgave_Ge\TxT_Ge.txt
../data/rich_txt_martijn/Uitgave_K\TxT_K.txt
../data/rich_txt_martijn/Uitgave_L\TxT_L.txt
../data/rich_txt_martijn/Uitgave_O\TxT_O.txt
../data/rich_txt_martijn/Uitgave_W\TxT_W.txt
../data/rich_txt_martijn/Uitgave_Y\TxT_Y.txt
../data/rich_txt_martijn/Uitgave_Z\TxT_Z.txt


In [13]:
def abbr_to_xml(kind, solution):
    abbr, expan = "", ""

    if kind == "$":
        if "_" in solution:
            val, nr = solution.split("_")
            abbr = '<hi rend="capitalsize'+nr+'">'+val+'</hi>'
        else:
            abbr = '<hi rend="capitalsize1">'+solution+'</hi>'
        expan = abbr
    elif kind == "=":
        abbr = '<num type="roman">'+solution+'</num>'
        expan = abbr
    elif kind == "£":
        abbr = '<g ref="#slongbar"/>'
        expan = "<ex>"+solution+"</ex>"
    elif kind == "*":
        abbr = '<unclear>'+solution+'</unclear>'
        expan = abbr
    elif kind == "(":
        abbr = '<g ref="#bar"/>'
        expan = "<ex>"+solution+"</ex>"
    elif kind == "[":
        if solution == "ist":
            abbr = 'p<g ref="#bar"/>'
            expan = "<ex>"+solution+"</ex>"
        else:
            abbr = '<g ref="#apomod"/>'
            expan = "<ex>"+solution+"</ex>"
    elif kind == "{":
        if solution == "et" or solution == "iet" or solution == "at":
            abbr = '<g ref="#etfin"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "pro":
            abbr = '<g ref="#pflour"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "par" or solution == "paer":
            abbr = '<g ref="#pbardes"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "per":
            abbr = '<g ref="#pbardes"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "con" or solution == "com":
            abbr = '<g ref="#condes"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "us" or solution == 'as': # careful, -as abbreviated as ꝰ (-us) is very rare!
            abbr = '<g ref="#usmod"/>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "eit" or solution == 'ijt': # careful, -ijt abbreviated as 'etfin' is very rare!
            abbr = '<g ref="#etfin"/>'
            expan = "<ex>"+solution+"</ex>"
    elif kind == "%":
        if solution == "rv":
            abbr = '<hi rend="superscript">v</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "ri":
            abbr = '<hi rend="superscript">i</hi>'
        elif solution == "ur":
            abbr = '<hi rend="superscript">z</hi>'
        elif solution == "ue":
            abbr = '<hi rend="superscript">e</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "ro":
            abbr = '<hi rend="superscript">o</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "roe" or solution == 'rou':
            abbr = '<hi rend="superscript">o</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "ua":
            abbr = '<hi rend="superscript">a</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "ra" or solution == "aer": # careful, -aer/-ai abbreviated as superscript a is very rare!
            abbr = '<hi rend="superscript">a</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "re":
            abbr = '<hi rend="superscript">e</hi>'
            expan = "<ex>"+solution+"</ex>"
        elif solution == "eit" or solution == "iet":
            abbr = '<hi rend="superscript">t</hi>'
            expan = "<ex>"+solution+"</ex>"
        else:
            abbr = '<hi rend="superscript">'+solution+'</hi>'
            expan = "<ex>"+solution+"</ex>"
    return abbr, expan

In [14]:
def parse_abbrevs(word):
    # insert dummy boundary marker:
    w = boundaries.sub(r"|\1", word)
    if not "|" in w:
        return word
    abbr = ""
    expan = ""
    prev_kind = ""
    for part in w.split("|"):
        part = endings.sub("", part).strip()
        if not part:
            continue
        if (part[0] == "%" and prev_kind == "%") or \
            (part[0] == "$" and prev_kind == "$") or \
            (part[0] == "=" and prev_kind == "=") or \
            (part[0] == "*" and prev_kind == "*") or \
            (part[0] == "£" and prev_kind == "£") or \
            (part[0] in ")]}"):
            part = part[1:]
        if part:
            if part[0] in "({[%$=*£":
                kind = part[0]
                prev_kind = kind
                solution = part[1:]
                a, e = abbr_to_xml(kind, solution)
                abbr += a
                expan += e
            else:
                abbr += part
                expan += part
    if abbr != expan:
        abbr = "<abbr>"+abbr+"</abbr>"
        expan = "<expan>"+expan+"</expan>"
        return '<choice>'+abbr+expan+'</choice>'
    else:
        return abbr

In [48]:
import re

def convert_to_xml(file_path):
    
    file_name = os.path.basename(file_path)
    variant_name = file_name.split('.')[0].split('_')[1]
    print("Parsing...")
    print("\t", "Variant:", variant_name)
    
    martijns = {'_M1_':'Eerste Martijn', '_M2_': 'Tweede Martijn', '_M3_': 'Derde Martijn', '_Colofon_': 'Colofon'}
    
    #MAUD 
    lines = add_numbers(file_path)
    
#     lines = [line.strip() for line in open(file_path, 'r', encoding='utf-8').readlines() if line.strip()]
     
    xml = ""
        
    for item in lines:
        line = item[1]
        line_number = item[0]
           
        line = line.replace(">", "$").replace("<", "$")
        
        # gaps that are the result of damage are marked as [...] in the txt file; replace with € (which later gets replaced with the appropriate tag)
        line = line.replace("[...]", "€")
        
        # txt files have to start with a &-marker, indicating the folium
        if line.startswith("&"):
            curr_page_nr = line.replace("&", "")
            xml+='\n\n<pb xml:id="'+variant_name+'.f'+curr_page_nr+'" n="'+curr_page_nr+'"/>\n'
            line_counter = 0
            
        # within a page, a specific text (e.g. Eerste Martijn) can start; these are marked with an underscore (e.g. _M1_)
        # these texts are assigned to unique tags (e.g. <text n="1" xml:id="M1"> ... </text>, <text n="2" xml:id="M2"> ... </text> )
        elif line.startswith("_"):
            for key, value in martijns.items():
                if line == key:
                    mart_xml_id = key.replace("_", "")
                    xml+='\n\n<text n="'+str(value)+'" xml:id="'+mart_xml_id+'">\n<body>\n<p>'
            if line == "____": # 4 underscores in a txt-file mark the end of a text-element
                xml+=f'</p>\n</body>\n</text>\n'
         
        #MAUD
        elif line_number != 'no_content_line':
        #else:
            line_counter+=1
            trailer = ""
            if line.endswith("#"):
                trailer += '<choice><sic></sic><corr><c type="shy">-</c></corr></choice>'
                line = line[:-1]
            words = line.split()
            
            # MAUD 
            l_with_number = f'"/><l n="{line_number}">' if line_number != 'no_content_line' else '"/><l>'
            
            xml += '<lb n="'+str(line_counter)+'" xml:id="'+variant_name+'f'+str(curr_page_nr)+'.'+str(line_counter)+l_with_number # '"/><l>' #<l> is NEW!
            for i, word in enumerate(words):
                if word in string.punctuation:
                    xml += '<pc>'+word+'</pc> '
                else:
                    xml += parse_abbrevs(word)+" "
            xml = xml.strip()
            xml = xml.replace("C|", '<g ref="#para"/>')
            xml+=trailer
            xml+="</l>" ##NEW!
            xml+="\n"
            xml = xml.replace("€", '<damage><gap></gap></damage>')
    
    header = open("header.txt", 'r').read()
    footer = open("footer.txt", 'r').read()
    with open("../data/xml_martijn/xml_"+variant_name+".xml", "w+", errors="replace") as F: #!!!!!! unknown unicode chars get replaced (?)
            F.write(header+xml+footer)

In [49]:
for file_path in file_paths:
    convert_to_xml(file_path)

Parsing...
	 Variant: A
[['no_content_line', '&112rb&'], ['no_content_line', '_M1_'], ['no_content_line', ''], ['001', '<V_4>Vaphene m[ar]tin hoe salt gaen'], ['002', '<S>al die w[eer]elt hiet la(n)ghe stae(n)'], ['003', '<I>n dus crancke(n) loue'], ['004', '<S>o moet v%ro%uwe v[er]eere saen'], ['005', 'Sond[er] twifel ende waen'], ['006', '<R>umen he[er]en houe'], ['007', '<I>c zie de(n) valsche(n) wel ontfaen'], ['008', '<D>ie de he[er]en cone(n) dwaen'], ['009', '<E>n(de) plucke(n) vande(n) stoue'], ['010', '<E>n(de) ic zie de(n) rechte(n) slaen'], ['011', '<B>eede bespotte(n) en(de) beuaen'], ['012', '<A>lse die meese in de cloue'], ['013', '<R>echts offene god v[er]scroue'], ['014', '<H_1>oe langhe sal ghedoghe(n) dit'], ['015', '<G>od die alle dinc besit'], ['016', '<I>n sine heerscapie'], ['017', '<D>at die goede vleeuwet en(de) bit'], ['018', '<E>n(de) he(m) ne doech no dat no dit'], ['019', '<D>at hi hiet ghedie'], ['no_content_line', ''], ['no_content_line', '&112va&'], ['no_